In [1]:
#python のバージョンの表示
import sys
print(sys.version)

3.12.11 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 12:58:53) [MSC v.1929 64 bit (AMD64)]


In [2]:
from ai import ai_match, ai2s

ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:11<00:00, 431.38it/s]

count     win    lose    draw
o        3027    1386     587
x        1397    2935     668
total    4424    4321    1255

ratio     win    lose    draw
o       60.5%   27.7%   11.7%
x       27.9%   58.7%   13.4%
total   44.2%   43.2%   12.6%



[('count',
  [{'win': 3027, 'lose': 1386, 'draw': 587},
   {'win': 1397, 'lose': 2935, 'draw': 668},
   {'win': 4424, 'lose': 4321, 'draw': 1255}],
  '7d'),
 ('ratio',
  [{'win': 0.6054, 'lose': 0.2772, 'draw': 0.1174},
   {'win': 0.2794, 'lose': 0.587, 'draw': 0.1336},
   {'win': 0.4424, 'lose': 0.4321, 'draw': 0.1255}],
  '7.1%')]

In [3]:
from marubatsu import Marubatsu

def judge(self):
    if self.move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 〇 の勝利の判定
    if self.is_winner(Marubatsu.CIRCLE):
        return Marubatsu.CIRCLE
    # × の勝利の判定
    elif self.is_winner(Marubatsu.CROSS):
        return Marubatsu.CROSS   
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [4]:
from mbtest import test_judge

test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [5]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:04<00:00, 1111.64it/s]

count     win    lose    draw
o        2960    1436     604
x        1438    2913     649
total    4398    4349    1253

ratio     win    lose    draw
o       59.2%   28.7%   12.1%
x       28.8%   58.3%   13.0%
total   44.0%   43.5%   12.5%



[('count',
  [{'win': 2960, 'lose': 1436, 'draw': 604},
   {'win': 1438, 'lose': 2913, 'draw': 649},
   {'win': 4398, 'lose': 4349, 'draw': 1253}],
  '7d'),
 ('ratio',
  [{'win': 0.592, 'lose': 0.2872, 'draw': 0.1208},
   {'win': 0.2876, 'lose': 0.5826, 'draw': 0.1298},
   {'win': 0.4398, 'lose': 0.4349, 'draw': 0.1253}],
  '7.1%')]

In [6]:
def judge(self):
    if self.move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(self.last_turn):
        return self.last_turn
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [7]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [8]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:03<00:00, 1463.72it/s]

count     win    lose    draw
o        2948    1381     671
x        1476    2899     625
total    4424    4280    1296

ratio     win    lose    draw
o       59.0%   27.6%   13.4%
x       29.5%   58.0%   12.5%
total   44.2%   42.8%   13.0%



[('count',
  [{'win': 2948, 'lose': 1381, 'draw': 671},
   {'win': 1476, 'lose': 2899, 'draw': 625},
   {'win': 4424, 'lose': 4280, 'draw': 1296}],
  '7d'),
 ('ratio',
  [{'win': 0.5896, 'lose': 0.2762, 'draw': 0.1342},
   {'win': 0.2952, 'lose': 0.5798, 'draw': 0.125},
   {'win': 0.4424, 'lose': 0.428, 'draw': 0.1296}],
  '7.1%')]

In [9]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [10]:
def is_winner(self, player):
    x, y = self.last_move
    if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
       self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
    # 左上から右下方向の判定
    if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [11]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [12]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:02<00:00, 2176.32it/s]

count     win    lose    draw
o        2907    1427     666
x        1449    2895     656
total    4356    4322    1322

ratio     win    lose    draw
o       58.1%   28.5%   13.3%
x       29.0%   57.9%   13.1%
total   43.6%   43.2%   13.2%



[('count',
  [{'win': 2907, 'lose': 1427, 'draw': 666},
   {'win': 1449, 'lose': 2895, 'draw': 656},
   {'win': 4356, 'lose': 4322, 'draw': 1322}],
  '7d'),
 ('ratio',
  [{'win': 0.5814, 'lose': 0.2854, 'draw': 0.1332},
   {'win': 0.2898, 'lose': 0.579, 'draw': 0.1312},
   {'win': 0.4356, 'lose': 0.4322, 'draw': 0.1322}],
  '7.1%')]

In [13]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    self.rowcount = {
        Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
        Marubatsu.CROSS: [0] * self.BOARD_SIZE,
    }
    self.colcount = {
        Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
        Marubatsu.CROSS: [0] * self.BOARD_SIZE,
    }
    self.diacount = {
        Marubatsu.CIRCLE: [0] * 2,
        Marubatsu.CROSS: [0] * 2,
    }
    
Marubatsu.restart = restart

In [14]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.colcount[self.last_turn][x] += 1
        self.rowcount[self.last_turn][y] += 1
        if x == y:
            self.diacount[self.last_turn][0] += 1        
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[self.last_turn][1] += 1        
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [15]:
def unmove(self):
    if self.move_count > 0:
        x, y = self.last_move
        self.colcount[self.last_turn][x] -= 1
        self.rowcount[self.last_turn][y] -= 1
        if x == y:
            self.diacount[self.last_turn][0] -= 1        
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[self.last_turn][1] -= 1           
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        
        self.last_move = self.records[-1]  
        
Marubatsu.unmove = unmove

In [16]:
def is_winner(self, player):
    x, y = self.last_move
    if self.rowcount[player][y] == self.BOARD_SIZE or \
       self.colcount[player][x] == self.BOARD_SIZE:
        return True
    # 左上から右下方向の判定
    if x == y and self.diacount[player][0] == self.BOARD_SIZE:
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.diacount[player][1] == self.BOARD_SIZE:
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [17]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [18]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:03<00:00, 1600.80it/s]

count     win    lose    draw
o        2988    1400     612
x        1434    2929     637
total    4422    4329    1249

ratio     win    lose    draw
o       59.8%   28.0%   12.2%
x       28.7%   58.6%   12.7%
total   44.2%   43.3%   12.5%



[('count',
  [{'win': 2988, 'lose': 1400, 'draw': 612},
   {'win': 1434, 'lose': 2929, 'draw': 637},
   {'win': 4422, 'lose': 4329, 'draw': 1249}],
  '7d'),
 ('ratio',
  [{'win': 0.5976, 'lose': 0.28, 'draw': 0.1224},
   {'win': 0.2868, 'lose': 0.5858, 'draw': 0.1274},
   {'win': 0.4422, 'lose': 0.4329, 'draw': 0.1249}],
  '7.1%')]

In [19]:
def judge(self):
    if self.move_count == 0:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(self.last_turn):
        return self.last_turn
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [20]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [21]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:03<00:00, 1642.92it/s]

count     win    lose    draw
o        2928    1443     629
x        1442    2900     658
total    4370    4343    1287

ratio     win    lose    draw
o       58.6%   28.9%   12.6%
x       28.8%   58.0%   13.2%
total   43.7%   43.4%   12.9%



[('count',
  [{'win': 2928, 'lose': 1443, 'draw': 629},
   {'win': 1442, 'lose': 2900, 'draw': 658},
   {'win': 4370, 'lose': 4343, 'draw': 1287}],
  '7d'),
 ('ratio',
  [{'win': 0.5856, 'lose': 0.2886, 'draw': 0.1258},
   {'win': 0.2884, 'lose': 0.58, 'draw': 0.1316},
   {'win': 0.437, 'lose': 0.4343, 'draw': 0.1287}],
  '7.1%')]

In [22]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    
Marubatsu.restart = restart

In [23]:
def is_winner(self, player):
    x, y = self.last_move
    if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
       self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
    # 左上から右下方向の判定
    if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [24]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [25]:
def unmove(self):
    if self.move_count > 0:
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        self.last_move = self.records[-1]  

Marubatsu.unmove = unmove

In [26]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [27]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:03<00:00, 1309.87it/s]

count     win    lose    draw
o        2936    1405     659
x        1433    2919     648
total    4369    4324    1307

ratio     win    lose    draw
o       58.7%   28.1%   13.2%
x       28.7%   58.4%   13.0%
total   43.7%   43.2%   13.1%



[('count',
  [{'win': 2936, 'lose': 1405, 'draw': 659},
   {'win': 1433, 'lose': 2919, 'draw': 648},
   {'win': 4369, 'lose': 4324, 'draw': 1307}],
  '7d'),
 ('ratio',
  [{'win': 0.5872, 'lose': 0.281, 'draw': 0.1318},
   {'win': 0.2866, 'lose': 0.5838, 'draw': 0.1296},
   {'win': 0.4369, 'lose': 0.4324, 'draw': 0.1307}],
  '7.1%')]